<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: spec2, extract_2d step for mos prism

<span style="color:red"> **Instruments Affected**</span>: NIRSpec 

Tested on CV3 data

### Table of Contents
<div style="text-align: left"> 

<br> [Imports](#imports_ID) <br> [Introduction](#intro_ID) <br> [Testing Data Set](#data_ID) <br> [Run the JWST pipeline and assign_wcs validation tests](#pipeline_ID): [FS Full-Frame test](#FULLFRAME), [FS ALLSLITS test](#ALLSLITS), [MOS test](#MOS) <br> [About This Notebook](#about_ID)<br> [Results](#results) <br>

</div>

<a id="imports_ID"></a>
# Imports
The library imports relevant to this notebook are aready taken care of by importing PTT.

* astropy.io for opening fits files
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot

NOTE: This notebook assumes that the pipeline version to be tested is already installed and its environment is activated.

To be able to run this notebook you need to install nptt. 

If all goes well you will be able to import PTT.

[Top of Page](#title_ID)

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
import shutil
data_dir = TemporaryDirectory()

os.chdir(data_dir.name)

In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /grp/crds/cache


In [3]:
import warnings
import psutil
from astropy.io import fits

# Only print a DeprecationWarning the first time it shows up, not every time.
with warnings.catch_warnings():
    warnings.simplefilter("once", category=DeprecationWarning)
    import jwst
    from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
    from jwst.assign_wcs.assign_wcs_step import AssignWcsStep
    from jwst.msaflagopen.msaflagopen_step import MSAFlagOpenStep
    from jwst.extract_2d.extract_2d_step import Extract2dStep

# The latest version of NPTT is installed in the requirements text file at:
# /jwst_validation_notebooks/environment.yml

# import NPTT
import nirspec_pipe_testing_tool as nptt

# To get data from Artifactory
from ci_watson.artifactory_helpers import get_bigdata


In [4]:
# Print the versions used for the pipeline and NPTT

pipeline_version = jwst.__version__
nptt_version = nptt.__version__

print("Using jwst pipeline version: ", pipeline_version)
print("Using NPTT version: ", nptt_version)

Using jwst pipeline version:  1.8.2
Using NPTT version:  2.0.1


<a id="intro_ID"></a>
# Test Description

We compared Institute's pipeline product of the assign_wcs step with our benchmark files, or with the intermediary products from the ESA pipeline, which is completely independent from the Institute's. The comparison file is referred to as 'truth'. We calculated the relative difference and expected it to be equal to or less than computer precision:  relative_difference = absolute_value( (Truth - ST)/Truth )  <= 1x10^-7. 

For the test to be considered PASSED, every single slit (for FS data), slitlet (for MOS data) or slice (for IFU data) in the input file has to pass. If there is any failure, the whole test will be considered as FAILED. 

The code for this test can be obtained at: https://github.com/spacetelescope/nirspec_pipe_testing_tool/blob/master/nirspec_pipe_testing_tool/calwebb_spec2_pytests/auxiliary_code/check_corners_extract2d.py. Multi Object Spectroscopy (MOS), the code is in the same repository but is named ```compare_wcs_mos.py```, and for Integral Field Unit (IFU) data, the test is named ```compare_wcs_ifu.py```.
The input file is defined in the variable ```input_file``` (see section [Testing Data Set and Variable Setup](#data_ID)).

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/extract_2d/main.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/extract_2d


### Results

If the test **PASSED** this means that all slits, slitlets, or slices individually passed the test. However, if ony one individual slit (for FS data), slitlet (for MOS data) or slice (for IFU data) test failed, the whole test will be reported as **FAILED**.

### Calibration WG Requested Algorithm: 

A short description and link to the page: 
https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Path-Loss+Correction


### Defining Term
Acronymns used un this notebook:

pipeline: calibration pipeline

spec2: spectroscopic calibration pipeline level 2b

PTT: NIRSpec pipeline testing tool (https://github.com/spacetelescope/nirspec_pipe_testing_tool)


[Top of Page](#title_ID)

<a id="pipeline_ID"></a>
# Run the JWST pipeline and extract_2d validation tests

The pipeline can be run from the command line in two variants: full or per step.

Tu run the spec2 pipeline in full use the command: 

$ strun jwst.pipeline.Spec2Pipeline jwtest_rate.fits

Tu only run the extract_2d step, use the command:

$ strun jwst.extract_2d.Extract2dStep jwtest_previous_step_output.fits


These options are also callable from a script with the testing environment active. The Python call for running the pipeline in full or by step are:

$\gt$ from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

$\gt$ Spec2Pipeline.call(jwtest_rate.fits)
 
or
 
$\gt$ from jwst.extract_2d import Extract2dStep
 
$\gt$ Extract2dStep.call(jwtest_previous_step_output.fits)


PTT can run the spec2 pipeline either in full or per step, as well as the imaging pipeline in full. In this notebook we will use PTT to run the pipeline and the validation tests. To run PTT, follow the directions in the corresponding repo page.

[Top of Page](#title_ID)

<a id="data_ID"></a>
# Testing Data Set

All testing data is from the CV3 campaign. We chose these files because this is our most complete data set, i.e. all modes and filter-grating combinations.

Data used was for testing was only FS and MOS, since extract_2d is skipped for IFU. Data sets are:
- MOS_PRISM_CLEAR


[Top of Page](#title_ID)

In [5]:
testing_data = {
                                
                'mos_prism_clear':{
                                  'uncal_file_nrs1': 'mos_prism_nrs1_uncal.fits',
                                  'uncal_file_nrs2': 'mos_prism_nrs2_uncal.fits',
                                  'truth_file_nrs1': 'mos_prism_nrs1_extract_2d_truth.fits',
                                  'truth_file_nrs2': None,
                                  'msa_shutter_config': 'V0030006000104_msa.fits' }
                
               }


In [6]:
# define function to pull data from Artifactory
def get_artifactory_file(data_set_dict, detector):
    """This function creates a list with all the files needed per detector to run the test.
    Args:
        data_set_dict: dictionary, contains inputs for a specific mode and configuration
        detector: string, either nrs1 or nrs2
    Returns:
        data: list, contains all files needed to run test
    """
    files2obtain = ['uncal_file_nrs1', 'truth_file_nrs1', 'msa_shutter_config']
    data = []
    for file in files2obtain:
        data_file = None
        try: 
            if '_nrs' in file and '2' in detector:
                file = file.replace('_nrs1', '_nrs2')

            data_file = get_bigdata('jwst_validation_notebooks',
                                         'validation_data',
                                         'nirspec_data', 
                                         data_set_dict[file])
        except TypeError:
            data.append(None)
            continue

        data.append(data_file)

    return data

In [7]:
# Set common NPTT switches for NPTT and run the test for both detectors in each data set

# define benchmark (or 'truth') file
compare_assign_wcs_and_extract_2d_with_esa = False

# accepted threshold difference with respect to benchmark files
extract_2d_threshold_diff = 4

# define benchmark (or 'truth') file
esa_files_path, raw_data_root_file = None, None
compare_assign_wcs_and_extract_2d_with_esa = False

# Get the data
results_dict = {}
detectors = ['nrs1', 'nrs2']
for mode_config, data_set_dict in testing_data.items():
    for det in detectors:
        print('Testing files for detector: ', det)
        data = get_artifactory_file(data_set_dict, det)
        uncal_file, truth_file, msa_shutter_config = data
        print('Working with uncal_file: ', uncal_file)
        uncal_basename = os.path.basename(uncal_file)
        
        # Make sure that there is an assign_wcs truth product to compare to, else skip this data set
        if truth_file is None:
            print('No truth file to compare to for this detector, skipping this file.  \n')
            skip_file = True
        else:
            skip_file = False

        if not skip_file:       
            # Run the stage 1 pipeline 
            rate_object = Detector1Pipeline.call(uncal_file)
            # Make sure the MSA shutter configuration file is set up correctly
            if msa_shutter_config is not None:
                rate_object.meta.instrument.msa_metadata_file = msa_shutter_config
                print('MSA shuter config file: ', rate_object.meta.instrument.msa_metadata_file)

            # Run the stage 2 pipeline steps
            pipe_object = AssignWcsStep.call(rate_object)
            if 'mos' in uncal_basename.lower():
                pipe_object = MSAFlagOpenStep.call(pipe_object)
            extract_2d_object = Extract2dStep.call(pipe_object)

            # Run the validation test
            %matplotlib inline

            if 'mos' in uncal_file.lower():
                print('Running test for MOS...')
                result, _ = nptt.calwebb_spec2_pytests.auxiliary_code.check_corners_extract2d.find_MOSwindowcorners(
                                                                   extract_2d_object,
                                                                   msa_shutter_config,
                                                                   truth_file=truth_file,
                                                                   esa_files_path=esa_files_path,
                                                                   extract_2d_threshold_diff= extract_2d_threshold_diff)


        else:
            result = 'skipped'

        # Did the test passed 
        print("Did extract_2d ", mode_config, " validation test passed? ", result, "\n\n")
        rd = {uncal_basename: result}
        results_dict.update(rd)
        
        # close all open files
        psutil.Process().open_files()
        closing_files = []
        for fd in psutil.Process().open_files():
            if data_dir.name in fd.path:
                closing_files.append(fd)
        for fd in closing_files:
            try:
                print('Closing file: ', fd)
                open(fd.fd).close()
            except:
                print('File already closed: ', fd)



Testing files for detector:  nrs1


Working with uncal_file:  /internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpn2bz941e/mos_prism_nrs1_uncal.fits


2022-12-03 08:35:43,333 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /grp/crds/cache/references/jwst/jwst_nirspec_pars-detector1pipeline_0004.asdf


2022-12-03 08:35:43,355 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.


2022-12-03 08:35:43,357 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.


...

2022-12-03 08:36:36,609 - stpipe.Extract2dStep - INFO - Step Extract2dStep done


Running test for MOS...
Information from the 'truth' (or comparison) file 
Filename: /internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpn2bz941e/mos_prism_nrs1_extract_2d_truth.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     279   ()      
  1  SCI           1 ImageHDU        65   (437, 12)   float32   
  2  DQ            1 ImageHDU        11   (437, 12)   int32 (rescales to uint32)   
  3  ERR           1 ImageHDU        10   (437, 12)   float32   
  4  WAVELENGTH    1 ImageHDU         9   (437, 12)   float32   
  5  VAR_POISSON    1 ImageHDU         9   (437, 12)   float32   
  6  VAR_RNOISE    1 ImageHDU         9   (437, 12)   float32   
  7  SCI           2 ImageHDU        65   (424, 11)   float32   
  8  DQ            2 ImageHDU        11   (424, 11)   int32 (rescales to uint32)   
  9  ERR           2 ImageHDU        10   (424, 11)   float32   
 10  WAVELENGTH    2 ImageHDU         9 

None


from assign_wcs file  -->     Detector: NRS1   Grating: PRISM   Filter: CLEAR   Lamp: LINE4
 *** WARNING! MSA config file name given in PTT_config file does not match the MSAMETFL keyword in main header.


Working with slitlet: 3
Truth slit size    =  [437, 12]
Pipeline slit size =  437 11

 Corners for slitlet 3:  [x0, y0, x1, y1]
       Truth corners: [1028, 1972, 1465, 1984]
    Pipeline corners: [1028, 1973, 1465, 1984]
* Pytest PASSED: All corners match within the threshold.

Working with slitlet: 4
Truth slit size    =  [424, 11]
Pipeline slit size =  424 9

 Corners for slitlet 4:  [x0, y0, x1, y1]
       Truth corners: [1276, 33, 1700, 44]
    Pipeline corners: [1276, 35, 1700, 44]
* Pytest PASSED: All corners match within the threshold.


 *** Final result for extract_2d test will be reported as PASSED *** 

All slitlets PASSED extract_2d test.
Did extract_2d  mos_prism_clear  validation test passed?  True 


Closing file:  popenfile(path='/internal/data1/jenkins/workspace/Not

Working with uncal_file:  /internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpn2bz941e/mos_prism_nrs2_uncal.fits
No truth file to compare to for this detector, skipping this file.  

Did extract_2d  mos_prism_clear  validation test passed?  skipped 




In [8]:
# Quickly see if the test passed 

print('These are the final results of the tests: ')
for key, val in results_dict.items():
    if not isinstance(val, str):
        if val:
            val = 'PASSED'
        else:
            val = 'FAILED'
    print('{:<42} {:<8}'.format(key, val))



These are the final results of the tests: 
mos_prism_nrs1_uncal.fits                  PASSED  
mos_prism_nrs2_uncal.fits                  skipped 


<a id="about_ID"></a>
## About this Notebook
**Author:** Maria A. Pena-Guerrero, Staff Scientist II - Systems Science Support, NIRSpec
<br>**Updated On:** Sep/23/2022

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 